# Spatial filters

## Imports

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import sys
from scipy.signal import fftconvolve
from scipy.optimize import minimize

sys.path.insert(1, '../mechanistic_models')
from functions import create_loggabor, create_loggabor_fft, create_gauss_fft, kelly_csf
from experimental_data.exp_params import stim_params as sparams

## Parameters

In [ ]:
# Constant params
sigma_angleo = 0.2965            # from Schütt & Wichmann (2017)
ppd = sparams["ppd"]             # pixel resolution
tf = 2.5                         # temporal frequency

# Create SF axes
nX = int(sparams["stim_size"]*ppd)
fs = np.fft.fftshift(np.fft.fftfreq(nX, d=1./ppd))
fx, fy = np.meshgrid(fs, fs)

# Extents
vex = [-sparams["stim_size"]/2, sparams["stim_size"]/2,]*2
fex = [fs.min(), fs.max(),]*2

# Single-scale filter

## CSF (Kelly)
The basis for the single-scale filter was the spatiotemporal contrast sensitivity function as defined in Kelly (1979).
Since we will need to fit the isotropic filters to this CSF, let's create it first.

In [ ]:
# Optional: Plot one-dimensional csfs
axSet = {"yscale": 'log', "ylim": (0.003, 1), "yticks": (0.01, 0.1, 1), "yticklabels": (0.01, 0.1, 1),
         "xlabel": 'cpd', "xscale": 'log', "xticks": (0.1, 1, 10, 100), "xticklabels": (0.1, 1, 10, 100),
         "xlim": (0.1)}

# Plot csfs from Fig 14
w_all = np.array([2., 2.5, 13.5, 17., 23.])  # temporal frequencies
fig, ax = plt.subplots(1, 2, figsize=(8, 3))
for i in range(len(w_all)):
    csfKelly = kelly_csf(np.linspace(0.1, 100, 1000), tfs=[w_all[i],])
    csfKelly[csfKelly!=0] = 1. / csfKelly[csfKelly!=0]
    ax[0].plot(np.linspace(0.1, 100, 1000), csfKelly, '.-', label=str(w_all[i]) + ' Hz')

# Plot csf with our temporal characteristics
csf = kelly_csf(fs, tfs=[tf,])
csfPlot = 1. / csf[csf!=0]
ax[1].plot(fs[fs!=0], csfPlot, '.-', label=str(tf)+" Hz")
ax[0].set(title='kelly79', **axSet), ax[0].legend(), ax[0].invert_yaxis()
ax[1].set(title='ours', **axSet),    ax[1].legend(), ax[1].invert_yaxis()
plt.show()

In [ ]:
# Two-dimensional csf for our experiment
csf_2d = kelly_csf(np.sqrt((fy**2. + fx**2.)), tfs=[tf,])

plt.figure(figsize=(12, 3))
plt.subplot(121), plt.imshow(csf_2d, extent=fex), plt.colorbar()
plt.subplot(122), plt.plot(fs, csf), plt.plot(fs, csf_2d[int(nX/2),:]), plt.xlim(0)
plt.show()

In [ ]:
# Quick comparison with castleCSF
import scipy.io as sio
mat_contents = sio.loadmat("../heuristic_test/castleCSF_schmittwilken2024.mat")
matCSF = mat_contents["csf"]
matSF = mat_contents["fd"]
print(np.unique(matSF - np.sqrt(fx**2 + fy**2))) # are we using the same SFs?

plt.figure(figsize=(12, 3))
plt.subplot(121), plt.imshow(matCSF, extent=fex), plt.colorbar()
plt.subplot(122), plt.plot(fs, csf / csf.max()), plt.plot(fs, matCSF[int(nX/2),:] / matCSF.max()), plt.xlim(0)
plt.show()

## Oriented log-Gabor (OG)
Plot original best-fitting log-Gabor filter

In [ ]:
# Perform fitting
def run_loggabor(params):
    fo = params[0]; sigma_fo = params[1]
    loggabor_fft = create_loggabor_fft(fx, fy, fo, sigma_fo, 0, sigma_angleo)
    
    p1 = csf_2d[int(nX/2), int(nX/2)::] / csf_2d.max()
    p2 = loggabor_fft[int(nX/2), int(nX/2)::]
    return np.abs(p1 - p2).sum()

params0 = [3., 0.55]
bnds = ((0.5, 4.), (0.05, 0.95))

res = minimize(run_loggabor, params0, method='SLSQP', bounds=bnds)
print(res)

# Create and plot the loggabor filter with the above specifications
loggabor_fft = create_loggabor_fft(fx, fy, res["x"][0], res["x"][1], 0, sigma_angleo)

In [ ]:
# Parameters
fo = 2.64553923            # fitted to CSF
sigma_fo = 0.49796013      # fitted to CSF

In [ ]:
# Create oriented log-Gabor
loggabor_fft = create_loggabor_fft(fx, fy, fo, sigma_fo, 0, sigma_angleo)
_, loggabor = create_loggabor(fx, fy, fo, sigma_fo, 0, sigma_angleo)

plt.figure(figsize=(12, 3))
plt.subplot(131), plt.imshow(loggabor_fft, extent=fex), plt.title("2d Fourier")
plt.subplot(132), plt.imshow(loggabor, extent=vex), plt.title("2d space")
plt.subplot(133)
plt.plot(fs, csf/csf.max(), label='CSF')
plt.plot(fs, loggabor_fft[int(nX/2),:], label='CSF-loggabor')
plt.legend(), plt.xlabel("SF"), plt.title("1d Fourier"), plt.xlim(0)
plt.show()

## Isotropic log-Gabor

In [ ]:
# Create isotropic log-Gabor (set sigma_angleo to inf)
isologgabor_fft = create_loggabor_fft(fx, fy, fo, sigma_fo, 0, np.inf)
isologgabor, _ = create_loggabor(fx, fy, fo, sigma_fo, 0, np.inf)

plt.figure(figsize=(12, 3))
plt.subplot(141), plt.imshow(isologgabor_fft, extent=fex, cmap="coolwarm"), plt.title("2d Fourier")
plt.subplot(142), plt.imshow(isologgabor, extent=vex, cmap="coolwarm"), plt.title("2d space")
plt.subplot(143)
plt.plot(fs, csf/csf.max(), label='CSF')
plt.plot(fs, isologgabor_fft[int(nX/2),:], label='CSF-loggabor')
plt.legend(), plt.xlabel("SF"), plt.title("1d Fourier"), plt.xlim(0)
plt.subplot(144); plt.plot(isologgabor[int(nX/2),:]), plt.title("1d space")
plt.show()

# Multi-scale filters

## Oriented log-Gabor (OG)

In [ ]:
# Parameters
fos = [0.5, 3., 9.]              # center SFs of log-Gabor filters
sigma_fo = 0.5945                # from Schütt & Wichmann (2017)
nFilters = len(fos)

# Create oriented log-Gabor (set sigma_angleo to inf)
loggabor_fft = create_loggabor_fft(fx, fy, fos[-1], sigma_fo, 0, sigma_angleo)
loggabor, _ = create_loggabor(fx, fy, fos[-1], sigma_fo, 0, sigma_angleo)

plt.figure(figsize=(12, 3))
plt.subplot(141), plt.imshow(loggabor_fft, extent=fex), plt.title("2d Fourier")
plt.subplot(142), plt.imshow(loggabor, extent=vex), plt.title("2d space")
plt.subplot(143), plt.plot(fs, loggabor_fft[int(nX/2),:]), plt.title("1d Fourier"), plt.xlim(0)
plt.subplot(144); plt.plot(loggabor[int(nX/2),:]), plt.title("1d space"), #plt.xlim(nX/2, nX/2+10)
plt.show()

In [ ]:
plt.figure(figsize=(3,2))
for fo in fos[::-1]:
    loggabor_fft = create_loggabor_fft(fx, fy, fo, sigma_fo, 0, sigma_angleo)
    plt.plot(fs, loggabor_fft[int(nX/2),:], label=str(fo)+" cpd"); plt.xlim(0);
#plt.legend(); plt.savefig("loggabors.png", dpi=300)

## Isotropic log-Gabor

In [ ]:
# Create isotropic log-Gabor (set sigma_angleo to inf)
isologgabor_fft = create_loggabor_fft(fx, fy, fos[0], sigma_fo, 0, np.inf)
isologgabor, _ = create_loggabor(fx, fy, fos[0], sigma_fo, 0, np.inf)

plt.figure(figsize=(12, 3))
plt.subplot(141), plt.imshow(isologgabor_fft, extent=fex), plt.title("2d Fourier")
plt.subplot(142), plt.imshow(isologgabor, extent=vex), plt.title("2d space")
plt.subplot(143), plt.plot(fs, isologgabor_fft[int(nX/2),:]), plt.title("1d Fourier"), plt.xlim(0)
plt.subplot(144); plt.plot(isologgabor[int(nX/2),:]), plt.title("1d space"), #plt.xlim(nX/2, nX/2+10)
plt.show()